In [ ]:
!pip install pyspark
!pip install requests
!pip install pandas

In [ ]:
!pyspark --help

Usage: ./bin/pyspark [options]

Options:
  --master MASTER_URL         spark://host:port, mesos://host:port, yarn,
                              k8s://https://host:port, or local (Default: local[*]).
  --deploy-mode DEPLOY_MODE   Whether to launch the driver program locally ("client") or
                              on one of the worker machines inside the cluster ("cluster")
                              (Default: client).
  --class CLASS_NAME          Your application's main class (for Java / Scala apps).
  --name NAME                 A name of your application.
  --jars JARS                 Comma-separated list of jars to include on the driver
                              and executor classpaths.
  --packages                  Comma-separated list of maven coordinates of jars to include
                              on the driver and executor classpaths. Will search the local
                              maven repo, then maven central and any additional remote
                    

In [ ]:
!cat /proc/meminfo

MemTotal:       13290460 kB
MemFree:         8192256 kB
MemAvailable:   12105368 kB
Buffers:          420820 kB
Cached:          3615472 kB
SwapCached:            0 kB
Active:           852396 kB
Inactive:        3895708 kB
Active(anon):       1312 kB
Inactive(anon):   712200 kB
Active(file):     851084 kB
Inactive(file):  3183508 kB
Unevictable:           8 kB
Mlocked:               8 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               196 kB
Writeback:             0 kB
AnonPages:        711892 kB
Mapped:           326548 kB
Shmem:              1692 kB
KReclaimable:     203924 kB
Slab:             249740 kB
SReclaimable:     203924 kB
SUnreclaim:        45816 kB
KernelStack:        5840 kB
PageTables:        11932 kB
SecPageTables:         0 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6645228 kB
Committed_AS:    3038168 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       12200 kB
VmallocChunk:    

In [ ]:
import requests
import zipfile
import os
import glob

def descargar_y_descomprimir_anidados(url, nombre_archivo):
    """Descarga un archivo desde una URL y descomprime archivos ZIP anidados."""
    try:
        respuesta = requests.get(url, stream=True)
        respuesta.raise_for_status()

        with open(nombre_archivo, 'wb') as archivo:
            for bloque in respuesta.iter_content(chunk_size=8192):
                archivo.write(bloque)
        print(f"Descarga exitosa: {nombre_archivo}")

        # Descomprimir el ZIP principal
        if nombre_archivo.lower().endswith('.zip'):
            with zipfile.ZipFile(nombre_archivo, 'r') as archivo_zip:
                archivo_zip.extractall(path=os.path.dirname(nombre_archivo))
            print(f"Archivo ZIP principal descomprimido: {nombre_archivo}")

            # Descomprimir los ZIPs anidados
            directorio_base = os.path.dirname(nombre_archivo)
            patron_zip_anidados = os.path.join(directorio_base, '*.zip')
            archivos_zip_anidados = glob.glob(patron_zip_anidados)

            for zip_anidado in archivos_zip_anidados:
                if zip_anidado != nombre_archivo: # Evitar procesar el archivo zip principal de nuevo
                    with zipfile.ZipFile(zip_anidado, 'r') as archivo_zip_anidado:
                        archivo_zip_anidado.extractall(path=directorio_base)
                    print(f"Archivo ZIP anidado descomprimido: {zip_anidado}")

    except requests.exceptions.RequestException as e:
        print(f"Error durante la descarga: {e}")
    except zipfile.BadZipFile:
        print(f"Error: {nombre_archivo} o uno de los archivos zip internos no es un archivo zip válido.")
    except IOError as e:
        print(f"Error de E/S al escribir o descomprimir el archivo: {e}")

# Ejemplo de uso
url_archivo = "https://bit.ly/1Aoywaq" #@param {type:"string"}
nombre_archivo_local = "donation.zip" #@param {type:"string"}

descargar_y_descomprimir_anidados(url_archivo, nombre_archivo_local)

Descarga exitosa: donation.zip
Archivo ZIP principal descomprimido: donation.zip
Archivo ZIP anidado descomprimido: block_4.zip
Archivo ZIP anidado descomprimido: block_8.zip
Archivo ZIP anidado descomprimido: block_2.zip
Archivo ZIP anidado descomprimido: block_6.zip
Archivo ZIP anidado descomprimido: block_3.zip
Archivo ZIP anidado descomprimido: block_7.zip
Archivo ZIP anidado descomprimido: block_5.zip
Archivo ZIP anidado descomprimido: block_10.zip
Archivo ZIP anidado descomprimido: block_9.zip
Archivo ZIP anidado descomprimido: block_1.zip


In [ ]:
import pyspark

# Create a SparkSession
spark = pyspark.sql.SparkSession.builder.appName("MySparkApp").getOrCreate()

# Now you can use the 'spark' variable to read the CSV file
prev = spark.read.csv('block_*.csv')#llemos todos los csv que empiezan con "block...."
prev.show()

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
|  _c0|  _c1|         _c2|         _c3|         _c4|         _c5|    _c6|   _c7|   _c8|   _c9|   _c10|    _c11|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
| 3148| 8326|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|14055|94934|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|33948|34740|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|  946|71870|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|64880|71676|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|  

En este data set se muestra el match o coincidencia entre distintos pacientes, es decir pacientes que tienen algo en comun, ( nombre, edad, direecion, etc.) a cada coincidencia veremos el campo final con TRUE en caso de que coincida algo y false lo opuesto.

In [ ]:
parsed = spark.read.option("header", "true").option("nullValue","?").option("inferSchema", "true").csv("block_*.csv")

En el paso anterior creamos un objeto que parsea ( remplaza los nulos,etc.) el dataframe original y lo almacena en la variable "parsed"

In [ ]:
parsed.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



Ver las primeras 5 lineas

In [ ]:
parsed.show(5)

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|14055|94934|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|33948|34740|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|  946|71870|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|64880|71676|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--

In [ ]:
parsed.count()

5749132

Mostramos la cantidad de matches y no matches

In [ ]:
from pyspark.sql.functions import col
parsed.groupBy("is_match").count().orderBy(col("count").desc()).show()

+--------+-------+
|is_match|  count|
+--------+-------+
|   false|5728201|
|    true|  20931|
+--------+-------+



Resumen del DF

In [ ]:
summary = parsed.describe()
summary.show()

+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|summary|              id_1|              id_2|      cmp_fname_c1|      cmp_fname_c2|       cmp_lname_c1|       cmp_lname_c2|            cmp_sex|             cmp_bd|             cmp_bm|            cmp_by|            cmp_plz|
+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|  count|           5749132|           5749132|           5748125|            103698|            5749132|               2464|            5749132|            5748337|            5748337|           5748337|            5736289|
|   mean| 33324.48559643438| 66587.43558331935|0.7129024704425707| 0.900017671890335|0.3156278193076

Pasamos el resumen a un DF de pandas

In [ ]:
 summary_p = summary.toPandas()
 print(summary_p)

  summary                id_1                id_2        cmp_fname_c1  \
0   count             5749132             5749132             5748125   
1    mean   33324.48559643438   66587.43558331935  0.7129024704425707   
2  stddev  23659.859374487987  23620.487613269706  0.3887583596162788   
3     min                   1                   6                 0.0   
4     max               99980              100000                 1.0   

         cmp_fname_c2         cmp_lname_c1         cmp_lname_c2  \
0              103698              5749132                 2464   
1   0.900017671890335  0.31562781930763056  0.31841283153174366   
2  0.2713176105782331   0.3342336339615803   0.3685670662006655   
3                 0.0                  0.0                  0.0   
4                 1.0                  1.0                  1.0   

               cmp_sex               cmp_bd               cmp_bm  \
0              5749132              5748337              5748337   
1    0.955001381078048

De esta manera podriamos trabajar con pandas tal como lo conocemos

In [ ]:
!pyspark

Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/18 22:06:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/18 22:06:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.5.5
      /_/

Using Python version 3.11.11 (main, Dec  4 2024 08:55:07)
Spark context Web UI available at http://7aa07fa5a0a2:4041
Spark context available as 'sc' (master = local[*], app id = local-1742335590783).
SparkSession available as 'spark'.
>>> myRange = spark.range(1000).toDF("number")
>>> print(myRange)
DataFrame[number: bigint]
>>> exit()


In [ ]:
myRange = spark.range(1000).toDF("number")
myRange.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows

